<a href="https://colab.research.google.com/github/sevwal/BME362_image_analysis/blob/main/BME362_Image_Processing_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BME362: Image Processing**
We will be running a relatively simple image processing pipeline. As I'm not aware of your skill levels in coding, this script is meant to be fancy enough to excite you but simple enough to follow through easily - at least I hope so. Of course, I will be here to help you throughout the course should you have any questions!

We will make use of the open source functionalities of Python. One powerful aspect of Python is the easy install of the abundant packages and dependencies that suit your needs. And of course: the large community that codes all of this and makes it accessible to you for free :D We will mainly make use of the following packages:

*   [`OpenCV`](https://github.com/opencv/opencv/wiki) (Open Source Computer Vision Library) is a tool for computer vision and machine learning. It provides us with some powerful algorithms to process our images.
*   [`Cellpose`](https://github.com/MouseLand/cellpose) is a tool for image segmentation of cells and nuclei. It provides us with the ability to train a neural network for image segmentation. Don't worry, we won't go that deep in this short training and will make use of the pre-trained models cellpose provides to us. For more information, check out their [paper](https://www.biorxiv.org/content/10.1101/2022.04.01.486764v1).
*   [`scikit-image`](https://scikit-image.org/) an open-source image processing library for Python.



---



Remarks: This script was inspired by the documentation on [Cellpose](https://github.com/MouseLand/cellpose) and their example notebooks, workflows developed by Joel Lüthi ([Twitter](https://twitter.com/joel_luethi), [Github](https://github.com/jluethi)), as well as the notebook [Running cellpose 2.0 in colab with a GPU](https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/run_cellpose_2.ipynb#scrollTo=Q7c7V4yEqDc_).


## Workflow

The basic workflow here is as follows:

1.   Set up the environment for this script to run. This means we install and import all the necessary packages, and we download and set up our image files and their corresponding folder (paths).
2.   Segment the images into nuclei and cytoplasm using Cellpose. We will make use of pre-trained models so we can simply test out some parameters on segmentation.
3.   Use these segmentations to extract measurements. We extract measurements of size, area, signal intensity, and more for each cell individually.
4.   Analyse the data and generate some nice plots. This will allow us to answer the scientific question at hand.



# **Preliminaries**

## **Step 1:** Get your files in here
Due to file size limitations, we will change the workflow as it was previously planned. We will now download only the wells relevant to answering the question of this experiment: D05 (transfection, wild type) and E05 (transfection, mutant).

In order to do that, we will download the images directly from a Dropbox folder containing these images.import os
  if (not os.path.exists("/content/BME362_data")):
      os.mkdir("/content/BME362_data")

In [ ]:
import os

#create new folder for our data
if (not os.path.exists("/content/BME362_data")):
    os.mkdir("/content/BME362_data")

#download files as .zip
!wget https://www.dropbox.com/sh/tg683owkitc8bxo/AABi_1Z6gtW2_xzrwe5Ahco9a?dl=0 --content-disposition -P "/content/BME362_data"

#unzip .zip
!unzip "/content/BME362_data/round2_D05_E05.zip" -d "/content/BME362_data"

#delete .zip
!rm "/content/BME362_data/round2_D05_E05.zip"

## **Step 2:** Setup Cellpose and Import Libraries
Run the code below to install OpenCV and the GPU version of Cellpose.

After installation, you will get an error that your runtime has crashed. **Do not worry this is part of the plan!** <br>

<img src=https://static01.nyt.com/images/2016/08/05/us/05onfire1_xp/05onfire1_xp-superJumbo-v2.jpg
width=400>

In [ ]:
#!pip install git+https://www.github.com/mouseland/cellpose.git #to install development version
!pip install cellpose
!pip install torch torchvision torchaudio

#Fix opencv error: ImportError: cannot import name '_registerMatType' from 'cv2.cv2'
!pip install "opencv-python-headless<4.3"
exit(0) #Restart Runtime to use newly installed numpy

As with all Python packages, installing is not enough. We need to import (and thus load) them to make them available for us to use with the code below. We will need some basics and dependencies (Numpy, time, os sys, random), along with our plotting tool (matplotlib.pyplot), and some additional things.

In [ ]:
#disabled installation and import of mxnet as pytorch is the default neural net
import numpy as np
import time, os, sys, random
from urllib.parse import urlparse
import skimage.io
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse
import shutil

## **Step 3:** Check GPU access and download Cellpose Models
Next up, we will download the pre-trained models from cellpose. Cellpose comes with the following pre-trained models:

*   Cytoplasm (`cyto`)
*   Cytoplasm2 (`cyto2`)
*   Cytoplasm2_Omnipose (`cyto2_omni`)
*   Bacteroa_Omnipose (`bact_omni`)
*   Nuclei (`nuclei`)

`Cytoplasm` and `Cytoplasm2` are two differently trained cytoplasm models (remember that these models are neural networks!). We can use either one but depending on your cells one may perform better than the other. The model for `nuclei` is pretty solid, as nuclei shapes are very similar across cell types.

`Omnipose` uses a different algorithm to provide robust segmentation of elongated cell shapes (such as certain bacteria and other cells). We will not need this here.

We will also check for GPU access to speed up our process if that's available. If you want to know more about why graphical processing units are important when working with neural networks, check out this [website](https://towardsdatascience.com/what-is-a-gpu-and-do-you-need-one-in-deep-learning-718b9597aa0d).

In [ ]:
print ("Downloading Models")
from cellpose import models,core
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
END = '\033[0m'

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  #Warnings from the ZeroCost StarDist notebook
  print(BOLD+UNDERLINE+'You do not have GPU access.'+END)
  print('Did you change your runtime ?')
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')
  use_GPU=False
else:
  print(BOLD+UNDERLINE+"You have access to the GPU."+END+"\nDetails are:")
  print("*************************************************")
  !nvidia-smi
  use_GPU=True

print("*************************************************")
print("Libraries imported and configured")

## **Step 4:** Prepare your paths
In Step 1 you made sure your images can be accessed with this script. Now's the time to use them:

*   Enter the full path as `Input_Directory` below. You can simply copy the path from the file viewer on the left side and paste it here.
*   Provide the format of the image (PNG, TIF, JPEG, GIF, ...) as `image_format`.

Besides this, the code below will create three new folders:

*   Merged, where we will store merged images with all four channels.
*   Masks, where we will store the segmentation masks.
*   Output, where we will store measurements and example images.



In [ ]:
#Specify input directory
Input_Directory = "/content/BME362_data" #@param {type:"string"}
input_dir = os.path.join(Input_Directory, "") #adds separator to the end regardless if path has it or not

#Specify image format
image_format = "png" #@param {type:"string"}

#Create a new folder where we will store our mask(s)
save_dir = input_dir+"Masks/"
if not os.path.exists(save_dir):
  os.makedirs(save_dir)
print("Masks: ", save_dir)

#Create a new folder where we will store our merged images
merge_dir = input_dir+"Merged/"
if not os.path.exists(merge_dir):
  os.makedirs(merge_dir)
print("Merges: ", merge_dir)

#Create a new folder where we will store our output
out_dir = input_dir+"Output/"
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
print("Output: ", out_dir)

#### **Interlude: Image formats**
In general, image analysis is always only recommended for ***lossless image formats*** such as ***TIF*** or ***PNG***. Jpeg is considered a lossy format, as it deletes part of the information in your images to compress them more. For more information, Wikipedia's page on image compression is pretty good (https://en.wikipedia.org/wiki/Image_compression).

As you should have seen, we will always use naming conventions to orient ourselves in the gazillion of images we take. The simplest solution now is to just throw everything into one folder, read that into `Python` as a path, and then access the files through loops and changing pathnames. Like this, you never have to specify individual files, which is helpful if you're dealing with a lot of images.

#### **Also, this:**
If you're having a tough time or just really need something for your ADHD needs, let's make life a little easier, shall we? Head to `Settings` (the gear in the upper right corner), then click `Miscellaneous`, and use any combination of `Corgi mode`, `Kitty mode`, and/or `Crab mode`. („• ᴗ •„)

##**Step 5:** Merge images
For the following code, we need to merge the channels of our images into one image with multiple channels. This is because segmenting with cellpose works best when images are merged.

*   In the code itself, specify the `wells` and `channels` you are using in the following format ["A", "B", ...].

Now, unfortunately, we need to repeat this step later on. This is because cellpose works best with merged images, but does not like images which have more than 3 channels... So in essence: We will omit a channels we don't need for segmentation. Our images have DAPI in channel 1 and Succinimidyl in channel 3. We will undo and merge all 4 channels later on.

In [ ]:
#Specify wells
wells = ["D05", "E05"] #@param {type:"raw"}

#Specify number of sites
number_of_sites = 25  #@param {type:"integer"}

#Loop through every site for every well and merge the channels into one image.
#The channels are hardcoded, so please adjust the if there are more or less than 3 channels
print("Merging images..." )
for well in wells:
  for site in range(1,number_of_sites+1):
    ch1 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C01.png" % (well, site),-1)
    ch2 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C02.png" % (well, site),-1)
    #ch3 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C03.png" % (well, site),-1)
    ch4 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C04.png" % (well, site),-1)
    merged = cv2.merge([ch1, ch2, ch4])

    cv2.imwrite(input_dir + "Merged/" +
                "29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01_merged.png" % (well, site), merged)
print("Merging finished.")

#### Interlude: Looping through folders and naming things
You might notice that the code in **step 5** was written in a way such that we loop through the folder containing our raw images and then merge all images that belong to the same well and site. Now you might realize why the naming conventions for image data are so useful. If metadata information is already in the filename, we can easily access and process files using just the filenmae.

This is why coding makes life so much easier for us in image analysis: We can create functions that are automatically and equally applied to all of our data!


## **Step 6**: Read and Load all the images, check a merged image
In this step we will check our images. As always, we would like to check if everything is set up correctly before blindly heading into later steps. So this will allow us to see if we are working with the correct images and double check the channel names for later. The code below will select a random image from the folder you provided and display all the channels of it.

**Note:** *We do not need to do anything here*, we can simply run this code as  all necessary variables are already specified. ٩(◕‿◕)۶


In [ ]:
# r=root, d=directories, f = files
files=[]

for r, d, f in os.walk(merge_dir):
    for fil in f:
      if (image_format):
        if fil.endswith(image_format):
          files.append(os.path.join(r, fil))
      else:
        files.append(os.path.join(r, fil))
    break #only read the root directory; can change this to include levels
if(len(files)==0):
  print("Number of images loaded: %d." %(len(files)))
  print("Cannot read image files. Check if folder has images")
else:
  print("Number of images loaded: %d." %(len(files)))


#Read and store all images
imgs=[]

for f in files:
  im=cv2.imread(f,cv2.IMREAD_UNCHANGED)
  imgs.append(im)
nimg = len(imgs)

#Load example image. This image is chosen randomly
print("Example Image:")

random_idx = random.choice(range(len(imgs)))
x=imgs[random_idx]
n_dim=len(x.shape)
file_name=os.path.basename(files[random_idx])

print(file_name+" has "+str(x.shape[2])+" channels.")


channel_image=x.shape[2]
fig, axs = plt.subplots(1, channel_image,figsize=(12,5))
print("Image: %s" %(file_name))
for channel in range(channel_image):
    axs[channel].imshow(x[:,:,channel])
    axs[channel].set_title('Channel '+str(channel+1),size=5)
    axs[channel].axis('off')
fig.tight_layout()

# **Nuclear Segmentation**

## **Step 7:** Specify parameters for the nuclear segmentation
We now have to specify the parameters for the segmentation of the nuclei. So what's needed?

*   **Channel to run segmentation on**. Which channel did you use to save the DAPI stain in your merged image?
*   **Model Type**. What model type do we need to use?
*   **Diameter**. How large are the nuclei in *pixels*, approximately? If you enter 0, cellpose will automatically estimate them. You will also be prompted to specify it later on, as well.



In [ ]:
#Specify Channel
Channel_for_segmentation="1" #@param[1,2,3]
segment_channel=int(Channel_for_segmentation)

#Specify Model
Model_Choice = "Nuclei" #@param ["Cytoplasm","Cytoplasm2", "Cytoplasm2_Omnipose", "Bacteria_Omnipose", "Nuclei"]
model_choice=Model_Choice

print("Using model ",model_choice)

if model_choice=="Cytoplasm":
  model_type="cyto"
elif model_choice=="Cytoplasm2":
  model_type="cyto2"
elif model_choice=="Cytoplasm2_Omnipose":
  model_type="cyto2_omni"
elif model_choice=="Bacteria_Omnipose":
  model_type="bact_omni"
  diameter = 0
elif model_choice=="Nuclei":
  model_type="nuclei"

#Diameter of cell (pixels):
Diameter =  0#@param {type:"number"}
diameter=Diameter

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type=model_type)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
#channels = [[2,3], [0,0], [0,0]]

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended)
# diameter can be a list or a single number for all images
if diameter is 0:
  diameter = None
  print("\nDiameter is set to None. The size of the cells will be estimated on a per image basis")

## **Step 8:** Test the nuclear segmentation
Next up, we want to check out how we're doing so far. Naturally, you don't want to run your code for all of your images only to then realise that you ran it with weird parameters. So, in order to do that, we need to specify one of the images we want to use for testing and give some additional parameters.



*   `Image_Number`: Which of your images you want to use for the test (starts at 1).
*   `Diameter`: The diameter of what you'd like to segment (roughly). This was optional before but is highly useful now.
*   `Flow_Threshold`: Maximum error for the "flows" for each mask. **Increase** to discard more cells, e.g. in range from (0.1, 3.0), or set to 0.0 to turn off so no cells are discarded.
*   `Cell_Probability_Threshold`: The probability threshold of a pixel being part of a cell. **Decrease** to include more pixels or **increase** to include fewer, e.g. in range from (-6, 6).

Naturally, we would not expect to get the parameters perfect in one go. Moreover, this depends on our cells, the experiment, and many more. So please go ahead and **fine-tune these parameters until you are satisfied with your segmentation**.

*Can you see any nuclei that were not segmented? Why do you think that is? Could you improve it by tweaking the parameters above?*


In [ ]:
from skimage.util import img_as_ubyte

#Specify Image Number
Image_Number =  2#@param {type:"number"}
Image_Number-=1 #indexing starts at zero

#Specify Diameter
Diameter = 40#@param {type:"number"}
diameter=Diameter

#Specify Flow_Threshold
Flow_Threshold = 0.8 #@param {type:"slider", min:0.0, max:1.1, step:0.1}
flow_threshold=Flow_Threshold

#Specify Cell_Probability_Threshold (Using slider to restrict values)
Cell_Probability_Threshold=0 #@param {type:"slider", min:-6, max:6, step:1}
cellprob_threshold=Cell_Probability_Threshold


#Channels = [cytoplasm, nucleus]
Use_nuclear_channel = False
if model_choice not in "Nucleus":
  if Use_nuclear_channel:
    channels=[segment_channel,nuclear_channel]
  else:
    channels=[segment_channel,0]
else: #nucleus
  channels=[segment_channel,0]

if diameter is 0:
  diameter = None
if Image_Number is -1:
  Image_Number=0
  #print("Image_Number is set to zero, opening first image.")
try:
    image = imgs[Image_Number]
except IndexError as i:
   print("Image number does not exist",i)
   print("Actual no of images in folder: ",len(imgs))
print("Image: %s" %(files[Image_Number]))
img1=imgs[Image_Number]

masks, flows, styles, diams = model.eval(img1, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)

# DISPLAY RESULTS
from cellpose import plot
maski = masks
flowi = flows[0]

#convert to 8-bit if not so it can display properly in the graph
if img1.dtype!='uint8':
  img1=img_as_ubyte(img1)

fig = plt.figure(figsize=(24,8))
plot.show_segmentation(fig, img1[:,:,segment_channel-1], maski, flowi, channels=channels)
plt.tight_layout()
plt.savefig(os.path.join(out_dir,"nuclei_segmentation_visualization.png"))
plt.show()

## **Step 9:** Run the nuclear segmentation
**Now it's up to you:** If you're happy, run the segmentation on all your nuclei and save the output as mask images with the regular filename but "`_NUC_MASK`" at the end. To do that, simply run the code below. It may take a couple of seconds, so go and stretch a bit, get some fresh air, and stay hydrated while your computer does the work.

Onwards to the cytoplasm!

In [ ]:
print("Running segmentation on channel %s" %(segment_channel))
print("Using the model: ",model_choice)
if diameter is None:
  print("Diameter will be estimated from the image/s")
else:
  print(f"Cellpose will use a diameter of {diameter}")

print(f"Using a flow threshold of: {flow_threshold} and a cell probability threshold of: {cellprob_threshold}")

#if too many images, it will lead to memory error.
#will evaluate on a per image basis
#masks, flows, styles, diams = model.eval(imgs, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)


#save images in folder with the diameter value used in cellpose
print("Segmentation Done. Saving Masks and flows now")
print("Save Directory is: ",save_dir)
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)


for img_idx, img in enumerate(imgs):
    file_name=os.path.splitext(os.path.basename(files[img_idx]))[0]
    print("\nSegmenting: ",file_name)
    mask, flow, style, diam = model.eval(img, diameter=diameter, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold, channels=channels)
    #save images in folder with the diameter value used in cellpose
    print("Segmentation complete . Saving Masks and flows")
    #Output name for masks
    mask_output_name=save_dir+file_name+"_NUC_MASK"+".png"
    print(mask_output_name)
    #Save mask as 16-bit in case this has to be used for detecting more than 255 objects
    mask=mask.astype(np.uint16)
    #Save flow as 8-bit
    skimage.io.imsave(mask_output_name, mask, check_contrast=False)

#Save parameters used in Cellpose
parameters_file=save_dir+"Cellpose_nuclear_parameters_used.txt"
outFile=open(parameters_file, "w")
outFile.write("CELLPOSE PARAMETERS\n")
outFile.write("Model: "+model_choice+"\n")
if diameter == 0:
  diameter = "Automatically estimated by cellpose"
outFile.write("Diameter: "+str(diameter)+"\n")
outFile.write("Flow Threshold: "+str(flow_threshold)+"\n")
outFile.write("Cell probability Threshold: "+str(cellprob_threshold)+"\n")
outFile.close()
print("\nSegmentation complete and files saved")

# **Cytoplasmic Segmentation**

## **Step 10:** Specify parameters for the cytoplasmic segmentation
We now have to specify the parameters for the segmentation of the cytoplasm. Again, let's take a look at the parameters we need:

*   **Model Type**. What model type do we need to use?
*   **Channel_for_segmentation**: Which channel did you use to save the Succs stain in your merged image?
*   **Use_nuclear_channel**: Before, this option was not given. If we segment the cytoplasm, we can choose to specify a nuclear channel, which will greatly help cytoplasmic segmentation.
*   **Diameter**. How large are the cells in *pixels*, approximately? Same as before, if you leave it at 0 cellpose will estimate them.

This is a repeat of what we did for the nuclei, with no major twists. The main difference here is that we choose a different model, and supply cellpose with the nuclear channel to help the cytoplasmic segmentation. *Food for thought: Why do you think that this helps?*

In [ ]:
#Specify Channel
Channel_for_segmentation="3" #@param[1,2,3]
segment_channel=int(Channel_for_segmentation)

Use_nuclear_channel= True #@param {type:"boolean"}
Nuclear_channel="1" #@param[1,2,3]
nuclear_channel=int(Nuclear_channel)

#Specify Model
Model_Choice = "Cytoplasm" #@param ["Cytoplasm","Cytoplasm2", "Cytoplasm2_Omnipose", "Bacteria_Omnipose", "Nuclei"]
model_choice=Model_Choice

print("Using model ",model_choice)

# Diameter of cell (pixels):
Diameter =  0#@param {type:"number"}
diameter=Diameter

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type=model_type)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
#channels = [[2,3], [0,0], [0,0]]

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended)
# diameter can be a list or a single number for all images
if diameter is 0:
  diameter = None
  print("\nDiameter is set to None. The size of the cells will be estimated on a per image basis")

## **Step 11:** Test the cytoplasmic segmentation
Very much analogous to what we did when we segmented the nuclei. I'm sure you'll still remember what these parameters meant! As before, do try **fine-tuning the parameters** and give cellpose a proper test run.

*Again, are there some cells that ar not or wrongly segmented? Do you see any reason why that could be (think of the shape of the cells, their cellular state, ...)?*

In [ ]:
from skimage.util import img_as_ubyte

#Specify Image Number
Image_Number =  2#@param {type:"number"}
Image_Number-=1 #indexing starts at zero

#Specify Diameter
Diameter = 50#@param {type:"number"}
diameter=Diameter

#Specify Flow_Threshold
Flow_Threshold = 0.9 #@param {type:"slider", min:0.0, max:1.1, step:0.1}
flow_threshold=Flow_Threshold

#Specify Cell_Probability_Threshold (Using slider to restrict values)
Cell_Probability_Threshold=0 #@param {type:"slider", min:-6, max:6, step:1}
cellprob_threshold=Cell_Probability_Threshold

if model_choice=="Cytoplasm":
  model_type="cyto"
elif model_choice=="Cytoplasm2":
  model_type="cyto2"
elif model_choice=="Cytoplasm2_Omnipose":
  model_type="cyto2_omni"
elif model_choice=="Bacteria_Omnipose":
  model_type="bact_omni"
  diameter = 0
elif model_choice=="Nuclei":
  model_type="nuclei"

# channels = [cytoplasm, nucleus]
if model_choice not in "Nucleus":
  if Use_nuclear_channel:
    channels=[segment_channel,nuclear_channel]
  else:
    channels=[segment_channel,0]
else: #nucleus
  channels=[segment_channel,0]

if diameter is 0:
  diameter = None
if Image_Number is -1:
  Image_Number=0
  #print("Image_Number is set to zero, opening first image.")
try:
    image = imgs[Image_Number]
except IndexError as i:
   print("Image number does not exist",i)
   print("Actual no of images in folder: ",len(imgs))
print("Image: %s" %(files[Image_Number]))
img1=imgs[Image_Number]

import cv2

masks, flows, styles, diams = model.eval(img1, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)

# DISPLAY RESULTS
from cellpose import plot
maski = masks
flowi = flows[0]

#convert to 8-bit if not so it can display properly in the graph
if img1.dtype!='uint8':
  img1=img_as_ubyte(img1)

fig = plt.figure(figsize=(24,8))
plot.show_segmentation(fig, img1[:,:,segment_channel-1], maski, flowi, channels=channels)
plt.tight_layout()
plt.savefig(os.path.join(out_dir,"cytoplasm_segmentation_visualization.png"))
plt.show()

## **Step 12:** Run the cytoplasmic segmentation
Once again we're back to running our segmentation on the cytoplasm. Again, if you're happy with cellpose's performance, run the segmentation on the cytoplasm. The masks will be saved in the MASK folder with the same filename but "`_CYT_MASK`" at the end.

In [ ]:
print("Running segmentation on channel %s" %(segment_channel))
print("Using the model: ",model_choice)
if diameter is None:
  print("Diameter will be estimated from the image/s")
else:
  print(f"Cellpose will use a diameter of {diameter}")

print(f"Using a flow threshold of: {flow_threshold} and a cell probability threshold of: {cellprob_threshold}")

#if too many images, it will lead to memory error.
#will evaluate on a per image basis
#masks, flows, styles, diams = model.eval(imgs, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)


#save images in folder with the diameter value used in cellpose
print("Segmentation Done. Saving Masks and flows now")
print("Save Directory is: ",save_dir)
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)


for img_idx, img in enumerate(imgs):
    file_name=os.path.splitext(os.path.basename(files[img_idx]))[0]
    print("\nSegmenting: ",file_name)
    mask, flow, style, diam = model.eval(img, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)
    #save images in folder with the diameter value used in cellpose
    print("Segmentation complete . Saving Masks and flows")
    #Output name for masks
    mask_output_name=save_dir+file_name+"_CYT_MASK"+".png"
    #Save mask as 16-bit in case this has to be used for detecting than 255 objects
    mask=mask.astype(np.uint16)
    #Save flow as 8-bit
    skimage.io.imsave(mask_output_name,mask, check_contrast=False)

#Save parameters used in Cellpose
parameters_file=save_dir+"Cellpose_cyto_parameters_used.txt"
outFile=open(parameters_file, "w")
outFile.write("CELLPOSE PARAMETERS\n")
outFile.write("Model: "+model_choice+"\n")
if diameter == 0:
  diameter = "Automatically estimated by cellpose"
outFile.write("Diameter: "+str(diameter)+"\n")
outFile.write("Flow Threshold: "+str(flow_threshold)+"\n")
outFile.write("Cell probability Threshold: "+str(cellprob_threshold)+"\n")
outFile.close()
print("\nSegmentation complete and files saved")

# *Re-merging*
Remember that we had to discard one of the channels for segmentation with cellpose? Well, now we can get it back! With the code below, we can re-merge all 4 channels of our image. This will allow us to make measurements for all channels simultaneously in the same image.

In [ ]:
#Loop through every site for every well and merge the channels into one image.
#The channels are hardcoded, so please adjust the if there are more or less than 3 channels
print("Merging images..." )
for well in wells:
  for site in range(1,number_of_sites+1):
    ch1 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C01.png" % (well, site),-1)
    ch2 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C02.png" % (well, site),-1)
    ch3 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C03.png" % (well, site),-1)
    ch4 = cv2.imread(input_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01C04.png" % (well, site),-1)
    merged = cv2.merge([ch1, ch2, ch3, ch4])

    cv2.imwrite(input_dir + "Merged/" +
                "29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01_merged.png" % (well, site), merged)
print("Merging finished.")

#I**mage Analysis**
So, what have we done so far? We have taken our raw images and, without any preprocessing, segmented our cells into both nuclei and cytoplasm using a clever package called `cellpose`. These segmentations (or masks) are now saved as .png images. So far so good.

In the coming part, we will not really be using the nuclear segmentations, as what you're trying to look at does not need them. We will continue with the cytoplasmic segmentations, clean them up, and then extract features per cell. This will allow us to get the data necessary for plotting our data and trying to answer the question we have.

As before, we first test before heading into applying our code to all the images. As such, the code below will once again randomly select one of your images, along with its segmentations.

Make sure to enter the channel of the images where you want to make measurements.

In [ ]:
#Chooses a random image from your input directory
files = []

for fil in os.listdir(merge_dir):
  f = os.path.join(merge_dir, fil)
  # checking if it is a file, this is to ignore any folders from being seen as a file
  if os.path.isfile(f):
      files.append(os.path.basename(f))

random_idx = random.choice(range(len(files)))

#Read signal and segmentation
signal = cv2.imread(merge_dir+files[random_idx], cv2.IMREAD_UNCHANGED)
cyt_seg = cv2.imread(save_dir + os.path.splitext(files[random_idx])[0] + "_CYT_MASK.png",-1)

## **Step 13:** Filtering
Now, when we extract information we want to be certain that the cells are **1) whole** (i.e. not cut off by the image border). If we don't, this will influence our analysis as all cell attributes (shape, size, signal,...) are affected by this. To do this, we will So in short, we don't want to measure shitty data, so we filter these cells first. Again, `skimage` has a wonderfully easy function for that called [`segmentation.clear_border`](https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.clear_border).

There are additional filtering steps we can take, for example to filter by size (or morphology). This removes any weird non-cellular objects (i.e. debris) that were segmented. However, as we are measuring stuff later on, this step becomes much easier in a bit. Moreover, cellpose is quite robust, if the parameters are chosen well. Other segmentation procedrures like the [watershed method](https://en.wikipedia.org/wiki/Watershed_%28image_processing%29) would require more cleanup.

In [ ]:
from skimage import segmentation

#Clear the labels of cells touching the border of the image (i.e. set to zero)
cyt_seg_filtered = segmentation.clear_border(cyt_seg)

## **Step 14:** Feature Extraction
So, we think everything looks good in our images, we have filtered our cells, how can we now extract information from these images?

If you take a look at your images in FIJI, Each cell or nucleus segmentation now has a different pixel value on that mask image. This allows us to extract all the coordinates of a cell by accessing only the pixels of that specific value.

Now, how we go about extracting this information on a per-cell-basis is up to us and there's many different options available. The best option depends on what you are doing and what you want from it. For this exercise, however, we will resort to a nice function of the excellent package `skimage` called [`measure.regionprops`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops).

We can provide this function with our *label image* (=mask image) and our *signal image* that we want to measure from. The code below will report how many properties are available for measurement. Now you need to decide which of them you need for your analysis (*hint: as with any function, consult its documentation!*).

In [ ]:
from skimage import measure

#List Available properties from measure.regionprops()
cyt_regionprops = measure.regionprops(cyt_seg_filtered, intensity_image=signal)
print("Available properties:")
print("  " + "\n  ".join(cyt_regionprops[0]))

Once you have selected some that might be of interest, put them in the corresponding spot in the code below. You might notice that the code looks a little bit different, but don't worry. It's essentially the same code as above (`measure.regionprops(label, intensity_image=image)`) but formats the output to a dataframe that is easier to use.

Looking at the output, what do you notice?

In [ ]:
import pandas as pd

cyt_measurements = pd.DataFrame(
    measure.regionprops_table(
        cyt_seg_filtered,
        intensity_image=signal,
        properties=['label', 'area', 'min_intensity', 'mean_intensity', 'max_intensity', 'solidity', 'minor_axis_length', 'major_axis_length', 'coords']
        )
).set_index('label')
cyt_measurements.head()

From the table above we can see the following:


*   **Label**: The label number of the segmented cell. Note that this is not unique accross wells (i.e. there are multiple cells with the same number in different images, but not in the same image). *Tip: You could make this unique if you combined well + site + label.*
*   **Area**: Size of the cells. Just consider it as a number, it technically is the number of pixels contained within the cell.
*   **Intensities**: The signal intensities for the different channels. Note that there are 4 different intensities corresponding to the 4 different channels. Since Python starts indexing at 0, this means that mean_intensity-0 corresponds to channel 1 (DAPI), mean_intensity-1 to channel 2, and so forth.
*   **Others**: There's more measurements in here but to see what they mean just check out the documentation of [measure.regionprops](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops).

## **Step 15:** Loop through all images
With all of this set, we can now do what we did above for all of our images. This essentially follows the same structure, except it is now in a loop. Additionally, we will save the measurements for each site into a CSV file.

**Note:** Make sure to modify the code below such that you use the same specifications and parameters as above.

In [ ]:
#Loop through every site for every well and merge the channels into one image.
#The channels are hardcoded, so please adjust the if there are more or less than 3 channels
for well in wells:
  for site in range(1,number_of_sites+1):
    signal = cv2.imread(merge_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01_merged.png" % (well, site), cv2.IMREAD_UNCHANGED)
    cyt_seg = cv2.imread(save_dir+"29092022_HEK_CD48_confocal_round2_%s_T0001F%03dL01A01Z01_merged_CYT_MASK.png" % (well, site),-1)

    #Clear the labels of cells touching the border of the image (i.e. set to zero)
    cyt_seg_filtered = segmentation.clear_border(cyt_seg)

    #Extract Features and store in pd.DataFrame
    cyt_measurements = pd.DataFrame(
        measure.regionprops_table(
            cyt_seg_filtered,
            intensity_image=signal,
            properties=['label', 'area', 'min_intensity', 'mean_intensity', 'max_intensity', 'eccentricity', 'minor_axis_length', 'major_axis_length', 'solidity', 'coords']
            )
    ).set_index('label')
    #Add Well and Site info to dataframe
    cyt_measurements["well"] = well
    cyt_measurements["site"] = site

    #Save measurements to CSV file
    cyt_measurements.to_csv(os.path.join(out_dir,
                                         "15092022_ZNFX1_testAB_%s_T0001F0%02dL01A01Z01_measurements.csv" % (well, site)))


To avoid loading all of the CSV files for our analysis, we can combine all of them into one. The code below quickly reads all your csv files that we generated previously and then combines them into one single file called `combined_measurements.csv`.

In [ ]:
import glob
all_filenames = [i for i in sorted(glob.glob(out_dir+"/*.csv"))]
combined_measurements = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_measurements = combined_measurements.reset_index()
combined_measurements.to_csv(os.path.join(out_dir, "combined_measurements.csv"), index=False, encoding='utf-8-sig')

# **Data Analysis**
You now have a large dataframe with many different variables. To answer our experiment question, we first think back to our experiment. **Remember**: What was the original goal of this experiment? What parameters do we need to answer this question? Where can you find these parameters?

From your programming and data analysis courses you should remember that weshould first do some quality checks on our data. For this, we'll do some histograms to check the distribution of variables such as area, or signal intensities. What we are looking for is nicely distributed data. Outliers, or bimodal distribution may give us a hint what we need to filter or what went wrong.

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks", palette="pastel")

dd = pd.read_csv(os.path.join(out_dir, "combined_measurements.csv"))

In [ ]:
# To check for differences between our conditions
# Compare the number of cells between wells
sns.displot(data=dd, x="well",
    facet_kws=dict(margin_titles=True))

Clearly, there's many more cells that were segmented in well E05. Seems peculiar, so we should keep that in mind.

In [ ]:
# To check for differences in segmentation between wells
# Compare the size of cells between wells
sns.displot(data=dd, x="area", col="well",
    facet_kws=dict(margin_titles=True),
)

We can see that this large difference in the number of cells between wells D05 and E05 stems from a large number of very small cells being present in well E05. We see that their size is close to 0, and barely any cells of that size are present in well D05. This hints that these cells were missegmented.

We don't know why, but it might be that cells were more confluent, there was a lot of debris, or our image quality was just bad. If you take a look at some images from well E05 you might notice that the signal intensity is pretty bad for some sites. This indicates to me that something went wrong during image acquisition, like the autofocus messing up.

Moreover, some cells might also be mitotic. For mitotic cells, we would expect them to have a high mean DAPI intensity, as they are dividing and have high DNA content. They are also a lot smaller and elongated (or: more elliptical than regular cells/nuclei).

Now for debris, we would expect that signal to be small and spotty, i.e. more perfectly round than a cell should be.

We can test our assumption with one more plot:

In [ ]:
sns.relplot(data=dd, x="area", y="eccentricity", hue="mean_intensity-0", alpha=0.4)

What have we done here? We plotted eccentricity vs area and used the mean DAPI intensity as a hue. Eccentricity is the ratio of the distance between two focal points over the length of the major axis. If eccentricity = 1 it is a perfect circle.

We can see three things:


1.   A large population of cells with a similarly low mean DAPI intensity (< 15'000) that is spread in both eccentricity and area.
2.   A second population of very small cells (area ~0) with high intensity (> 30'000).
3.   A third population of almost perfectly round cells (eccentricity == 1) with high intensity.

Recall what we said about mitotic cells and debris. Our second population probably represents mitotic cells, while the third corresponds to debris.

In our next step we will thus filter cells for eccentricity and area. Not only will we filter for small cells, we will also filter for very large cells (where several cells were combined into one, see histogram above).



In [ ]:
dd_filtered = dd[(dd["area"]>=650) & (dd["area"]<=7500) & (dd["eccentricity"]<=0.95)]

sns.displot(data=dd_filtered, x="area", col="well",
    facet_kws=dict(margin_titles=True),
)

sns.relplot(data=dd_filtered, x="area", y="eccentricity", hue="mean_intensity-0", alpha=0.4)

So, with those quality checks settled, let's dig deeper. First we might ask, well how many cells do we have where transfection actually worked? GFP was the signal we used to check if transfection worked. So now we can create a new variable called `transfection`. This will be a binary (0 = no, 1 = yes) for each cell depending on whether it reached a GFP threshold or not (i.e. was transfected or not).

In [ ]:
dd_filtered["transfected"] = np.where(dd_filtered["mean_intensity-1"]>=500, 1, 0) #1 if >=1000, 0 if <1000

print("Transfection efficiency:\n")

for well in wells:
  t = dd_filtered.shape[0] #No. Total
  p = dd_filtered[(dd_filtered["transfected"]==1) & (dd_filtered["well"]==well)].shape[0] #No. of transfected
  n = dd_filtered[(dd_filtered["transfected"]==0) & (dd_filtered["well"]==well)].shape[0] #No. of non-transfected
  print("Well %s" % (well))
  print("Total cells: %d\nTransfected: %d\nNon-transfected: %d" % (t,p,n))
  print("\nTransfection Efficiency: ", p/t*100, "%\n")


sns.displot(data=dd_filtered, x="mean_intensity-1">2000, col="well",
    facet_kws=dict(margin_titles=True),
)

In [ ]:
sns.boxplot(data=dd_filtered, x="well", y="mean_intensity-1")

In [ ]:
!zip -r /content/BME362_data.zip /content/BME362_data
from google.colab import files
files.download("/content/BME362_data.zip")